In [1]:
from sqlalchemy import create_engine
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv(verbose=True,
            dotenv_path='./.env')
engine = create_engine(f'postgresql://{os.getenv("USERNAME")}:{os.getenv("PASSWORD")}@{os.getenv("HOST")}/{os.getenv("DATABASE")}', 
                       connect_args={'options': '-csearch_path={}'.format('de')})

In [ ]:
drug_exposure = pd.read_sql_table('drug_exposure', engine)
condition_occurrence = pd.read_sql_table('condition_occurrence', engine)

# No.6


In [318]:
x = drug_exposure[['person_id', 'drug_concept_id', 'drug_exposure_start_date', 'drug_exposure_end_date', 'visit_occurrence_id']] # drug_exposure에서 설명상 주어진 columns

In [319]:
code = [3191208,36684827,3194332,3193274,43531010,4130162,45766052, 45757474,4099651,4129519,4063043,4230254,4193704,4304377,201826,3194082,3192767]
target_patients = condition_occurrence[condition_occurrence['condition_concept_id'].isin(code)] # 제 2형 당뇨 환자 정보

In [320]:
y = condition_occurrence[['person_id', 'condition_concept_id', 'condition_start_datetime', 'condition_end_datetime', 'visit_occurrence_id']] # condition_occures에서 설명상 주어진 columns

In [321]:
drug_pres = x.merge(y, 'inner') # person_id를 기준으로 join

In [489]:
from collections import defaultdict

In [532]:
patients = drug_pres.person_id.unique()
ans = defaultdict(int)
# 1. 환자별로 어떤 약을 처방 받았는지 패턴을 파악
# 2. 각 패턴별로 딕셔너리 구성
for patient in patients:
    test = drug_pres[drug_pres['person_id'] == patient]
    # 피벗 테이블을 사용해 날짜별로 모아주고
    # stack을 이용해 약품별로 펼침
    ty = pd.pivot_table(test[['drug_exposure_start_date', 'person_id', 'drug_concept_id']], index='drug_exposure_start_date', columns='drug_concept_id').stack(level=0)
    tx = ty.reset_index()
    tx = tx.set_index(tx['drug_exposure_start_date']).drop(['drug_exposure_start_date', 'level_1'], axis=1)
    tx.columns = map(str, tx.columns)
    # df.notna()와 column의 행렬곱을 통해 해당 약을 처방한 경우만 추가되도록 함
    # 원래 druf_concept_id가 int였기 때문에 string으로 바꿔주어야 했음
    # sort_index로 날짜 오름차순 정렬
    patient_record = pd.DataFrame(tx.notna().dot(tx.columns+(",")).str.rstrip(',')).sort_index()
    
    pattern = []
    # 2가지 이상 약을 처방한 경우 괄호로 묶어주고, 처방한 약의 id를 pattern리스트에 추가
    for index, pair in patient_record.iterrows():
        if len(pair.str.split(',')[0]) > 1:      
            pattern.append("("+pair.values[0]+")")
        else:
            pattern.append(pair.values[0])
    
    # 패턴을 나타내도록 ->로 join해주고 정답이 될 dict에 횟수 추가
    pattern = " -> ".join(pattern)
    ans[pattern] += 1

In [542]:
answer6 = pd.DataFrame.from_dict(ans, orient='index', columns=['count']).sort_values(by='count',ascending=False) # 테이블 형태로 만들기
answer6.head()

,count
1127433,40
1115171,18
40213154,14
40229134,12
1713671,12


In [543]:
answer6

,count
1127433,40
1115171,18
40213154,14
40229134,12
1713671,12
...,...
"(920300,40236824) -> 19133905 -> 40229134",1
"(1332419,1539411,19075601,19126352,40213154) -> 19133873 -> 19133873 -> (1332419,1539411,19075601,19126352) -> 1127433 -> 19019979 -> 1127433 -> (1545959,19133768)",1
"19019979 -> 40229134 -> (1511248,40169216)",1
19133873 -> 1115171,1
